In [1]:
# for domonkos to run on COLAB

import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/python/Computer_Vision/TDT4265_Project/')

Mounted at /content/drive


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.data import load_train_test_val, load_tee
from src.visualize import plot_loss_acc
from src.model import Unet2D
#from src.baseline_model import Unet2D
from src.train import train
from src.utils import save_result, to_cuda
from src.test import test
from src.metrics import acc_metric
from datetime import datetime

from params import *

import torch
from torch import nn

## Prepare data

In [4]:
train_data, test_data, valid_data = load_train_test_val(DATA_PARAMS, PREP_STEPS, TRAIN_TRANSFORMS)

DATASET: extracted_CAMUS

Items loaded: 1800 [training: 1600, test: 100, valid: 100]
RAW IMAGES: torch.Size([8, 1, 384, 384])
 GT IMAGES: torch.Size([8, 384, 384])



## Train

In [5]:
# MODEL: Unet2D (one input channel, 4 output channels)
unet = Unet2D(1, out_channels=4)
print(unet)

loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.Adam(unet.parameters(), lr=LEARN_RATE)

Unet2D(
  (conv1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Dropout2d(p=0.1, inplace=False)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Dropout2d(p=0.1, inplace=False)
    (7): 

In [ ]:
if LOAD: # Load pretrained model
    unet.load_state_dict(torch.load(MODEL_PATH+FILE_NAME))
    to_cuda(unet)
    print(f"The pre-trained model'{FILE_NAME}' has been loaded")
else:  # Start the training process
    start = datetime.now()
    train_loss, valid_loss, train_acc, valid_acc = train(unet, train_data, valid_data, loss_fn, opt, acc_metric, epochs=EPOCHS)

    end = datetime.now()
    print(f"Elapsed time is {str(end-start)}")

Epoch 1/1
----------


In [ ]:
if not LOAD:
    plot_loss_acc(train_loss, valid_loss, train_acc, valid_acc)

## Test

In [ ]:
print("TEST on the CAMUS dataset")
accuracy, average_dice, class_dice = test(unet, test_data, True)

In [ ]:
print(f"Test on TEE images")
tee_data = load_tee(DATA_PARAMS['base_path'], DATA_PARAMS['batch_size'])
accuracy_tee, average_dice_tee, class_dice_tee = test(unet, tee_data, True)

# Save model

In [ ]:
if not LOAD:
  # save the result
  msg = 'Test the saving function'

  file_name = 'Gauss_bringth_scale15_model'

  # Save the model
  save_result(unet, file_name, accuracy_tee, average_dice_tee, class_dice_tee, msg = msg)
else:
  print("The model was already trained")